In [1]:
import pandas as pd
import numpy as np
import networkx as nx

/var/folders/kj/d1xg4wx91kd1nwc9s2xdpz_r0000gn/T/ipykernel_42696/4014824597.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# load state data
state_df = pd.read_csv('../data/states.csv')

# drop all entries of Year_Semester including the word "Summer"
state_df = state_df[~state_df['Year_Semester'].str.contains('Summer')]

state_df

,id,Semester_Num,CSCI128,CSCI200,CSCI220,CSCI274,CSCI306,CSCI341,CSCI358,CSCI370,CSCI400,CSCI406,CSCI442,MATH111,MATH112,MATH213,MATH332,Year_Semester
0,3,10094,0,0,1,1,1,1,1,1,1,3,3,0,1,1,1,Fall 2018
1,3,10095,-1,-1,0,0,0,1,1,1,1,2,1,0,1,1,1,Spring 2019
3,3,10099,-1,-1,0,0,-1,0,1,-1,-1,2,0,0,1,1,0,Fall 2019
4,3,10100,-1,-1,0,0,-1,0,0,-1,-2,0,-1,0,1,1,0,Spring 2020
6,3,10104,-1,-1,0,0,-1,0,0,-1,-2,0,-1,0,1,1,0,Fall 2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84834,2186,10120,0,0,1,1,1,1,1,1,1,3,3,0,1,1,1,Spring 2024
84836,6481,10114,0,1,1,1,1,1,1,1,2,3,3,0,1,1,1,Fall 2022
84837,6481,10115,0,1,1,1,1,1,1,1,2,3,3,0,1,1,1,Spring 2023
84839,6481,10119,0,1,1,1,1,1,1,1,2,3,3,0,1,1,1,Fall 2023


In [3]:
print(state_df.columns)

Index(['id', 'Semester_Num', 'CSCI128', 'CSCI200', 'CSCI220', 'CSCI274',
       'CSCI306', 'CSCI341', 'CSCI358', 'CSCI370', 'CSCI400', 'CSCI406',
       'CSCI442', 'MATH111', 'MATH112', 'MATH213', 'MATH332', 'Year_Semester'],
      dtype='object')


In [4]:
classes = ['CSCI128', 'CSCI200', 'CSCI220', 'CSCI274',
       'CSCI306', 'CSCI341', 'CSCI358', 'CSCI370', 'CSCI400', 'CSCI406',
       'CSCI442', 'MATH111', 'MATH112', 'MATH213', 'MATH332']

In [5]:
min_values = [state_df[c].min() for c in classes]
max_values = [state_df[c].max() for c in classes]
global_min = min(min_values)
global_max = max(max_values)

# normalize the data
if (global_max - global_min) < 10: 
    for c in classes:
        state_df[c] = state_df[c] - global_min
else: 
    print("ERROR: signature range is too large")

# set variables to count 
eligible = -1 * global_min

In [6]:
# create column concatenating all columns in classes
state_df['signature'] = state_df[classes].apply(lambda x: ''.join(x.astype(str)), axis=1)

# get list of all signatures
signatures = state_df['signature'].unique()


In [8]:
# create two networks: one showing Fall to Spring transitions, another showing Spring to Fall transitions

G_fall_to_spring = nx.DiGraph()
G_spring_to_fall = nx.DiGraph()

graphs = [G_fall_to_spring, G_spring_to_fall]

# add nodes to graph
for G in graphs: 
    G.add_nodes_from(signatures)

    # add 0 weight edges between nodes
    for s1 in signatures:
        for s2 in signatures:
            G.add_edge(s1, s2, weight=0)
        G.add_edge(s2, s1, weight=0)

In [ ]:
for id in state_df['id'].unique(): 
    id_df = state_df[state_df['id'] == id]

    # get list in order of signatures
    sigs = id_df['signature'].tolist()

    # add 1 to weight of edge between each pair of signatures
    for i in range(len(sigs) - 1): 

        # fall to spring
        if id_df['Year_Semester'].iloc[i] == 'Fall':
            G_fall_to_spring[sigs[i]][sigs[i+1]]['weight'] += 1
        # spring to fall
        else:
            G_spring_to_fall[sigs[i]][sigs[i+1]]['weight'] += 1
    

In [ ]:
graph_to_probability_matrix = {}

for G in graphs: 

    # based on graph, create Markov chain matrix
    weighted_adj_matrix = nx.to_numpy_array(G, weight='weight')

    # Normalize the weighted adjacency matrix
    row_sums = weighted_adj_matrix.sum(axis=1, keepdims=True)
    # Avoid division by zero for rows that sum to zero
    row_sums[row_sums == 0] = 1
    normalized_adj_matrix = weighted_adj_matrix / row_sums

    graph_to_probability_matrix[G] = normalized_adj_matrix

# Calculating State Changes

In [141]:
def get_state_vector(year_semester):
    spring23_df = state_df[state_df['Year_Semester'] == year_semester]

    # reduce spring23_df to include counts of signatures
    spring23_df = spring23_df.groupby('signature').size().reset_index(name='count')

    # Ensure the DataFrame is ordered to match the Markov matrix
    df_ordered = spring23_df.set_index('signature').reindex(signatures).reset_index()

    # Now, df_ordered['count'] can be directly used as the state vector
    state_vector = df_ordered['count'].fillna(0).to_numpy()

    return state_vector

In [142]:
def count_taken(vector): 
    # create count dictionary
    count_dict = {cl:0 for cl in classes}

    for i in range(len(vector)):
        sig = signatures[i]
        for index,digit in enumerate(str(sig)): 
            if int(digit) < eligible: 
                count_dict[classes[index]] += vector[i]

    return count_dict

In [165]:
def get_enrollment(year_semester): 
    NUM_LIST = ['Cap', 'Act', 'Rem', 'WL Cap','WL Act','WL Rem','XL Cap','XL Act','XL Rem']
    CORE_LIST = ['128', '200', '220', '274', '306', '341', '358', '370', '400', '406', '442']

    # load schedule data
    name = year_semester.split(" ")[0] + year_semester.split(" ")[1][2:]
    schedule_df = pd.read_csv(f'../schedules/{name}.csv')

    # remove 102 entries
    df = schedule_df[schedule_df['Title'] != 'INTRODUCTION TO COMPUTER SCIENCE - LAB']

    # change CrseNum to string
    df['CrseNum'] = df['CrseNum'].astype(str)

    # replace equivalent classes
    df['CrseNum'] = df['CrseNum'].replace('101', '128')
    df['CrseNum'] = df['CrseNum'].replace('261', '200')
    df['CrseNum'] = df['CrseNum'].replace('262', '220')

    # replace equivalent course names
    df['Title'] = df['Title'].replace('INTRODUCTION TO COMPUTER SCIENCE', 'COMPUTER SCIENCE FOR STEM')
    df['Title'] = df['Title'].replace('PROGRAMMING CONCEPTS', 'FOUNDATIONAL PROGRAMMING CONCEPTS & DESIGN')
    df['Title'] = df['Title'].replace('DATA STRUCTURES', 'DATA STRUCTURES AND ALGORITHMS')

    # filter to only CrseNum in CORE_LIST
    df = df[df['CrseNum'].isin(CORE_LIST)]

    # set all columns in NUM_LIST to int
    for col in NUM_LIST:
        df.loc[:,col] = df[col].astype(int)

    # group by class and year
    core_df = df.groupby(['CrseNum','Title']).sum().reset_index()

    return core_df

In [181]:
def find_actual_pass(year_semester): 
    actual_df = pd.read_csv(f'../data/combined.csv')
    actual_df = actual_df[actual_df['Year_Semester'] == year_semester][['Class','Taken']]

    # convert actual_df into dictionary
    actual_dict = actual_df.set_index('Class').to_dict()['Taken']

    if 'CSCI370' not in actual_dict: 
        actual_dict['CSCI370'] = 0

    return actual_dict

In [182]:
semesters_list = ['Fall 2021', 'Spring 2022', 'Fall 2022', 'Spring 2023', 'Fall 2023']

In [196]:
# create storage df
results_df = pd.DataFrame(columns=['year_semester', 'class', 'difference', 'true', 'predicted', 'actual enrollment'])

# just cs classes
cs_classes = [cl for cl in classes if cl[:4] == 'CSCI']

# get predictions
for index in range(1, len(semesters_list)):

    # get predicted passes
    state_vector = get_state_vector(semesters_list[index-1])
    predicted_fall23 = state_vector @ normalized_adj_matrix
    predicted_count_dict = count_taken(predicted_fall23)

    # get actual passes
    actual_count_dict = find_actual_pass(semesters_list[index])

    # get previous semester
    prev_count_dict = count_taken(state_vector)

    # get enrollment
    enrollment = get_enrollment(semesters_list[index])

    # find difference prev and true
    true_comparison_dict = {cl: actual_count_dict[cl] for cl in cs_classes}

    # find difference prev and predicted
    predicted_comparison_dict = {cl: predicted_count_dict[cl] - prev_count_dict[cl] for cl in cs_classes}

    # find error between true and predicted
    error_dict = {cl: actual_count_dict[cl] - predicted_comparison_dict[cl] for cl in cs_classes}

    # find difference true and predicted
    print(f"PREDICTIONS FOR {semesters_list[index].upper()}")
    for cl in cs_classes: 
        enroll_value = enrollment[enrollment['CrseNum'] == cl[4:]]['Act'].values[0]

        # add results to results_df
        results_df = pd.concat([results_df, pd.DataFrame([[semesters_list[index], cl, error_dict[cl], true_comparison_dict[cl], predicted_comparison_dict[cl], enroll_value]], 
                                                         columns=['year_semester', 'class', 'difference', 'true', 'predicted', 'actual enrollment'])])


/var/folders/kj/d1xg4wx91kd1nwc9s2xdpz_r0000gn/T/ipykernel_34300/1959220545.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CrseNum'] = df['CrseNum'].astype(str)
/var/folders/kj/d1xg4wx91kd1nwc9s2xdpz_r0000gn/T/ipykernel_34300/1959220545.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CrseNum'] = df['CrseNum'].replace('101', '128')
/var/folders/kj/d1xg4wx91kd1nwc9s2xdpz_r0000gn/T/ipykernel_34300/1959220545.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

PREDICTIONS FOR SPRING 2022
PREDICTIONS FOR FALL 2022
PREDICTIONS FOR SPRING 2023
PREDICTIONS FOR FALL 2023


In [197]:
results_df

,year_semester,class,difference,true,predicted,actual enrollment
0,Spring 2022,CSCI128,-114.764624,482,596.764624,552
0,Spring 2022,CSCI200,-35.497628,168,203.497628,226
0,Spring 2022,CSCI220,-93.868130,131,224.868130,312
0,Spring 2022,CSCI274,-51.568360,127,178.568360,151
0,Spring 2022,CSCI306,49.985326,194,144.014674,197
0,Spring 2022,CSCI341,34.410736,179,144.589264,139
0,Spring 2022,CSCI358,48.774084,149,100.225916,163
0,Spring 2022,CSCI370,-94.869145,0,94.869145,0
0,Spring 2022,CSCI400,-38.638695,105,143.638695,117
0,Spring 2022,CSCI406,64.160539,149,84.839461,176


In [199]:
# calculate pass rate
results_df['pass rate'] = (results_df['true'] + 0.00000001 )/ (results_df['actual enrollment'] + 0.00000001)

In [200]:
# print average difference between true and predicted
for cl in cs_classes:
    print(f"{cl}")
    print(f"\tAVG. DIFF: {results_df[results_df['class'] == cl]['difference'].mean()}")

    # find percent error compared to true
    print(f"\tAVG. ERROR: {100 * results_df[results_df['class'] == cl]['difference'].mean() / results_df[results_df['class'] == cl]['actual enrollment'].mean()}%")

    # calculate historical pass rate for class
    print(f"\tAVG. PASS: {results_df[results_df['class'] == cl]['pass rate'].mean()}")

    print()

CSCI128
	AVG. DIFF: -7.498629564026373
	AVG. ERROR: -1.1997807302442198%
	AVG. PASS: 0.9083904450819381

CSCI200
	AVG. DIFF: -6.493811977834525
	AVG. ERROR: -2.2646249268821363%
	AVG. PASS: 0.7945962783465665

CSCI220
	AVG. DIFF: -79.80887982575518
	AVG. ERROR: -34.58673015200658%
	AVG. PASS: 0.7605488994074237

CSCI274
	AVG. DIFF: -46.39898478909555
	AVG. ERROR: -27.133909233389208%
	AVG. PASS: 0.9102777706806608

CSCI306
	AVG. DIFF: -9.224172757775193
	AVG. ERROR: -5.52345674118275%
	AVG. PASS: 0.9676896622612021

CSCI341
	AVG. DIFF: -32.57653952437974
	AVG. ERROR: -22.085789508054063%
	AVG. PASS: 0.9635260853463679

CSCI358
	AVG. DIFF: 36.385053046586876
	AVG. ERROR: 19.99178738823455%
	AVG. PASS: 0.9174970692395796

CSCI370
	AVG. DIFF: -65.2954882047668
	AVG. ERROR: -114.05325450614288%
	AVG. PASS: 0.9955886885502467

CSCI400
	AVG. DIFF: 8.463565430398859
	AVG. ERROR: 6.045403878856328%
	AVG. PASS: 0.9361125368917942

CSCI406
	AVG. DIFF: 47.49354483856604
	AVG. ERROR: 27.7740028295

You should be predicting how many ELIGIBLE there are, not how many will take a class